In [1]:
import findspark
from pyspark import SparkContext
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.window import *
import pandas as pd
findspark.init()
spark = SparkSession.builder.getOrCreate()

In [20]:
path = "C:/Users/garci/Documents/Wizeline/all_data.csv"
df = spark.read.csv(path, header=True, inferSchema= True)

In [21]:
df.select("estado").distinct().toPandas()

,estado
0,QUINTANA ROO
1,NUEVO LEÓN
2,SINALOA
3,TABASCO
4,BAJA CALIFORNIA
5,TLAXCALA
6,COAHUILA DE ZARAGOZA
7,None
8,"ESQ. SUR 125"""
9,CHIAPAS


In [65]:
#bad_states = ['None', ' ESQ. SUR 125"', ' COL. EDUARDO GUERRA', "estado"]
df.filter(col("estado") == "None").show(3)
df.filter(col("estado") == ' ESQ. SUR 125"').show(3)
df.filter(col("estado") == ' COL. EDUARDO GUERRA').show(3)
df.filter(col("estado") == "estado").show(3)


+--------+------------+-----+---------+--------+------+-------------+---------------+----+---------------+---------+------+---------+-------+--------+
|producto|presentacion|marca|categoria|catalogo|precio|fechaRegistro|cadenaComercial|giro|nombreComercial|direccion|estado|municipio|latitud|longitud|
+--------+------------+-----+---------+--------+------+-------------+---------------+----+---------------+---------+------+---------+-------+--------+
+--------+------------+-----+---------+--------+------+-------------+---------------+----+---------------+---------+------+---------+-------+--------+

+--------+------------+-----+---------+--------+------+-------------+---------------+----+---------------+---------+------+---------+-------+--------+
|producto|presentacion|marca|categoria|catalogo|precio|fechaRegistro|cadenaComercial|giro|nombreComercial|direccion|estado|municipio|latitud|longitud|
+--------+------------+-----+---------+--------+------+-------------+---------------+----+---

In [ ]:
path = "C:/Users/garci/Documents/Wizeline/all_data.csv"
df = spark.read.option("quote", "\"").option("escape", "\"").csv(path, header=True, inferSchema= True)

In [ ]:
df.select("estado").distinct().toPandas()

,estado
0,QUINTANA ROO
1,NUEVO LEÓN
2,SINALOA
3,TABASCO
4,BAJA CALIFORNIA
5,TLAXCALA
6,COAHUILA DE ZARAGOZA
7,None
8,CHIAPAS
9,VERACRUZ DE IGNACIO DE LA LLAVE


## 1. How many commercial chains are monitored, and therefore, included in this database?

## 1. How many commercial chains are monitored, and therefore, included in this database?

In [23]:
df.agg(countDistinct("cadenaComercial").alias("Number of commercial chains")).collect()[0]

Row(Number of commercial chains=705)

In [29]:
df = df.filter(col("estado") != "estado")
df.agg(countDistinct("cadenaComercial").alias("Number of commercial chains")).collect()[0]

Row(Number of commercial chains=704)

In [ ]:

df = df.filter(~col("estado").isin(bad_states))

## 2. What are the top 10 monitored products by State?

In [43]:
df.filter(col("estado") == ' ESQ. SUR 125"').select("direccion").show(3, truncate = False)

+----------------------+
|direccion             |
+----------------------+
|"ORIENTE 100 """"A""""|
|"ORIENTE 100 """"A""""|
|"ORIENTE 100 """"A""""|
+----------------------+
only showing top 3 rows



In [46]:
df.filter(col("estado") == ' ESQ. SUR 125"').show(3)

+------------------+------------+-----+---------+--------+------+--------------------+--------------------+--------------------+---------------+--------------------+--------------+----------------+--------------------+---------+
|          producto|presentacion|marca|categoria|catalogo|precio|       fechaRegistro|     cadenaComercial|                giro|nombreComercial|           direccion|        estado|       municipio|             latitud| longitud|
+------------------+------------+-----+---------+--------+------+--------------------+--------------------+--------------------+---------------+--------------------+--------------+----------------+--------------------+---------+
|PAN BLANCO BOLILLO|       PIEZA|  S/M|      PAN| BASICOS|   1.5|2012-04-10 00:00:...|PANADERIAS TRADIC...|PANADERIA Y PASTE...|  PAN BARCELONA|"ORIENTE 100 """"...| ESQ. SUR 125"|DISTRITO FEDERAL|IZTACALCO        ...|19.395734|
|PAN BLANCO BOLILLO|       PIEZA|  S/M|      PAN| BASICOS|   1.5|2012-07-10 00:00:..

In [47]:
df.filter(col("estado") == ' COL. EDUARDO GUERRA').show(3)

+----------------+-------------+------+--------------------+--------+------+--------------------+---------------+--------------------+--------------------+--------------------+--------------------+-----------+--------------------+--------+
|        producto| presentacion| marca|           categoria|catalogo|precio|       fechaRegistro|cadenaComercial|                giro|     nombreComercial|           direccion|              estado|  municipio|             latitud|longitud|
+----------------+-------------+------+--------------------+--------+------+--------------------+---------------+--------------------+--------------------+--------------------+--------------------+-----------+--------------------+--------+
|  HARINA DE MAIZ|PAQUETE 1 KG.|MASECA|TORTILLAS Y DERIV...| BASICOS|  10.9|2016-04-11 00:00:...| BODEGA AURRERA|TIENDA DE AUTOSER...|BODEGA AURRERA SU...|"CALZ. LAZARO CAR...| COL. EDUARDO GUERRA| CP. 27280"|COAHUILA DE ZARAGOZA| TORREÓN|
|  HARINA DE MAIZ|PAQUETE 1 KG.| MINSA|T

In [6]:
prod_counts = df.groupby("estado","producto").count()

window = (Window
        .partitionBy(prod_counts["estado"])
        .orderBy(prod_counts["count"].desc()))

res = prod_counts.select(col("*"), rank().over(window).alias("rank")).filter(col("rank") <= 10).orderBy(col("estado"), col("count").desc())

In [7]:
res.show()

+---------------+--------------------+-----+----+
|         estado|            producto|count|rank|
+---------------+--------------------+-----+----+
| AGUASCALIENTES|                 FUD|12005|   1|
| AGUASCALIENTES|   DETERGENTE P/ROPA|10188|   2|
| AGUASCALIENTES|LECHE ULTRAPASTEU...| 9824|   3|
| AGUASCALIENTES|             SHAMPOO| 9654|   4|
| AGUASCALIENTES|            REFRESCO| 9481|   5|
| AGUASCALIENTES|         DESODORANTE| 8859|   6|
| AGUASCALIENTES|    JABON DE TOCADOR| 8517|   7|
| AGUASCALIENTES|      CHILES EN LATA| 7946|   8|
| AGUASCALIENTES|             YOGHURT| 7401|   9|
| AGUASCALIENTES|            MAYONESA| 7173|  10|
|BAJA CALIFORNIA|            REFRESCO|37243|   1|
|BAJA CALIFORNIA|   DETERGENTE P/ROPA|23395|   2|
|BAJA CALIFORNIA|                 FUD|19967|   3|
|BAJA CALIFORNIA|             SHAMPOO|19123|   4|
|BAJA CALIFORNIA|    JABON DE TOCADOR|18348|   5|
|BAJA CALIFORNIA|      CHILES EN LATA|16676|   6|
|BAJA CALIFORNIA|            GALLETAS|15873|   7|


## 3. Which is the commercial chain with the highest number of monitored products?

## 4. Use the data to find an interesting fact.

#### Repeated municipio names
At first I was trying to analyze the distribution of expensive products in cities when I found that there are repeated municipio names in some states, it can be seen that "ZACATECAS" appear twice, there is probably some extra space in the data entry.  
Fixing this error would imply going state by state to find repeated names and fixing them manually.

## 5. What are the lessons learned from this exercise?

Must ensure the data quality of the database before performing analysis. Failing to do this might invalidate an analysis and we might not catch this error before its too late.

## 6. Can you identify other ways to approach this problem? Explain.

 If the file would be larger it would be impossible to run this program, I suspect that many are not able to load the complete file, to solve this we would need to process it in a distributed manner.
 Downloading the file to databricks and then running the analysis using PySpark APIs would allows to handle the whole file.

We can even use the [Koalas](https://koalas.readthedocs.io/en/latest/) library to reuse the pandas code on spark although it would run slower than using only Spark APIs.


